# Análise Exploratória com Pandas
## Estudando as empresas Listadas na bolsa de Valores B3

Dados extraídos do portal [Fundamentus](https://www.fundamentus.com.br/resultado.php) no dia 01/02/2024

In [ ]:
import pandas as pd
import pprint
import seaborn as  sns
import matplotlib.pyplot as plt
import re
from unicodedata import normalize

pd.set_option('display.max_columns', None)

In [ ]:
# helpers

def format_column_names(text):


    
    string_step1 = text.replace(".", "_")
    
    string_step2 = string_step1.replace(' ', "_")
    
    string_step3 = string_step2.replace('/', '_')

    string_step4 = string_step3.replace('__', '_')

    if '_' == string_step4[-1]:

        return normalize('NFKD', string_step4[:-1]).encode('ASCII', 'ignore').decode('ASCII')
    
    else:

        return normalize('NFKD', string_step4).encode('ASCII', 'ignore').decode('ASCII')



## ATIVIDADE 1

In [ ]:
#Leirura do arquivo

data = pd.read_csv("./dados/empresas_bolsas.csv", sep=";")


In [ ]:
# 7 primeiras linhas do dataset

data.head(7)

In [ ]:
# 5 últimas linhas do dataset

data.tail(5)

In [ ]:
# Estrutura do DataFrame

linhas = data.shape[0]
colunas = data.shape[1]
print(f"O DataFrame Possui {linhas} linhas e {colunas} colunas")

In [ ]:
# Explorando as colunas do DataFrame

print(data.info())

In [ ]:
# Identificando as colunas do DataFrame

pp = pprint.PrettyPrinter(indent=4)
colunas_Df = data.columns
pp.pprint(list(colunas_Df))

columns_exclude = [
    'PAPEL',
    'DIV.YIELD',
    'MRG EBIT',
    'MRG. LÍQ.',
    'ROIC',
    'ROE',
    'CRESC. REC.5A',
]

numerical_columns = list(data.columns)

for i in columns_exclude:
    numerical_columns.remove(i)

percentual_number_columns = [
    'DIV.YIELD',
    'MRG EBIT',
    'MRG. LÍQ.',
    'ROIC',
    'ROE',
    'CRESC. REC.5A'
]

In [ ]:
# transformando os valores percentuais para float

for i in percentual_number_columns:
    data[i] = data[i].str[:-1]
    data[i] = data[i].str.replace(',', '.')

    for j in range(len(data[i])):
        list_index = []
        value = data[i][j]

        for index in range(len(value)):
            if value[index] == '.':
                list_index.append(index)

        new_value = value[:list_index[-1]].replace('.', '') + value[list_index[-1]:]

        #data[i][j] = new_value
        data.loc[j, i] = new_value
        #print(new_value)
        #data[i].replace(j, new_value, inplace=True)
        #data.replace({i:{j:new_value}}, inplace=True) 
        #data[i] = data[i].replace(j, new_value)     
        #print(data[i][j])

for i in percentual_number_columns:
    data[i] = data[i].astype('float')
    data[i] = data[i]*0.01

In [ ]:
# transformando os dados numéricos em formato de String para tipo Float

for i in numerical_columns:
    if data[i].dtype == 'object':
        data[i] = data[i].str.replace(',', '.')
        data[i] = data[i].astype('float')

In [ ]:
data.head(10)

In [ ]:
data.tail(10)

In [ ]:
data.info()

## ATIVIDADE 2

In [ ]:
# Buscando Valores ausentes na tabela

data.isna().sum()

In [ ]:
# preenchendo valores ausentes com Zero

data.fillna(value=0, inplace=True)
data.isna().sum()

In [ ]:
# Ajustes dos nomes das colunas

columns = list(data.columns)
new_columns = {col: format_column_names(col) for col in columns}
data.rename(columns=new_columns, inplace=True)
data.head()

In [ ]:
# Média do DIVIDEND YIELD das empresas com MARGEM LÍQUIDA MAIOR QUE 10%

dividend_top = data.query("DIV_YIELD >= 0.06 and MRG_LIQ > 0.1")
dividend_top['DIV_YIELD'].describe()

In [ ]:
sns.histplot(dividend_top['DIV_YIELD'], bins=10)

In [ ]:
# percentual das empresas que possuem P/L maior que 4

df_PL_company = data.query("P_L > 4")
percentual_company = (df_PL_company.shape[0]/data.shape[0])*100
print(" {:.2f}% das empreas listadas tem P/L maior que 4".format(percentual_company, 2))

df_PL_company['P_L'].describe()

In [ ]:
# empresas com *LIQUIDEZ CORRENTE* acima de 1, TEM *ROE* acime da média das empreas e *DIVIDEND YIELD* maior ou igual a 6.

dividend_average = data['ROE'].mean()
company_top = data.query(f"ROE > {dividend_average} & DIV_YIELD >= 6")
company_top

In [ ]:
# Analisando valores do ROEs das empresas

data[['ROE', 'DIV_YIELD']]

In [ ]:

print(data['ROE'].describe())
sns.histplot(data['ROE'], bins=40)


In [ ]:
# Análise DIVIDEND YIELD

data[['PAPEL', 'DIV_YIELD']]

In [ ]:
# Ordenando o DataFrame com base em 'DIV_YIELD' em ordem decrescente
data_sorted = data.sort_values('DIV_YIELD', ascending=False)

# Selecionando as primeiras 10 linhas
data_top10 = data_sorted.head(20)

# Criando o gráfico de barras para os 10 maiores valores de 'DIV_YIELD'
plt.figure(figsize=(10,6))
sns.barplot(x='DIV_YIELD', y='PAPEL', data=data_top10[4:])
plt.title('Top 10 PAPEL por DIV_YIELD')
plt.xlabel('DIVIDEND YIELD')
plt.ylabel('PAPEL')
plt.show()


In [ ]:
papeis = list(data.PAPEL)

In [ ]:
papeis